# Cross Year Validation

### All the Function

In [ ]:
def return_label(df,end_year):
    return_dict={}
    companies=pd.DataFrame(df[str(end_year)].dropna())
    
    for cik, label in companies.iterrows():
        return_dict[cik]=label[str(end_year)]
        
    return return_dict

In [2]:
from tqdm import tqdm
def get_xy(corpus,labels):
    X=[]
    Y=[]
    for ix in tqdm(labels.keys()):
        if corpus.get(str(ix),""):
            X.append(corpus[str(ix)])
            Y.append(labels[ix])
    return X,Y

In [ ]:
import pickle 
import pandas as pd
import numpy as np
year = 2011

base_corpus=pickle.load(open(f"GPT/future/{year}_future_documents_embd.pkl","rb"))
rev_df=pd.read_csv(f"labels/{year}.csv",index_col=1)


labels = return_label(rev_df, year)
test_labels = return_label(rev_df, year+1)

filtered_dict = {}
for key in test_labels:
    if key in labels:
        filtered_dict[key] = test_labels[key]
test_labels = filtered_dict


X, Y = get_xy(base_corpus, labels)

X = np.array(X)
Y = np.array(Y)

X_test, Y_test = get_xy(base_corpus, test_labels)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

### Cross data check

In [60]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, precision_score, recall_score
import numpy as np

def do_train_10f(corpus, year):
    labels = return_label(rev_df, year)
    X, Y = get_xy(corpus, labels)
    
    X = np.array(X)
    Y = np.array(Y)

    map_ = np.array([False if np.isnan(i).any() else True for i in X])

    X = X[map_]
    Y = Y[map_]
    
    # change +5 to how many year a head do you want to test
    test_labels = return_label(rev_df, year+5)

    filtered_dict = {}
    for key in test_labels:
        if key in labels:
            filtered_dict[key] = test_labels[key]
    test_labels = filtered_dict

    X_test, Y_test = get_xy(base_corpus, test_labels)
    
    map_ = np.array([False if np.isnan(i).any() else True for i in X_test])

    X_test = np.array(X_test)
    Y_test = np.array(Y_test)
    
    X_test = X_test[map_]
    Y_test = Y_test[map_]

    # Initialize lists to store performance metrics for each fold
    lr_train_accs, lr_test_accs, svc_train_accs, svc_test_accs = [], [], [], []
    lr_prs, svc_prs = [], []
    lr_recals, svc_recals = [], []
    lr_f1s, svc_f1s = [], []
    ovr_aurocs, ovo_aurocs = [], []

    # Create a 10-fold stratified cross-validation splitter
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    for train_indices, test_indices in skf.split(X, Y):
        x_train, x_test = X[train_indices], X[test_indices]
        y_train, y_test = Y[train_indices], Y[test_indices]

        # Train-test split within the fold (0.1 test size)
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train)

        # Logistic Regression
        lr = LogisticRegression(n_jobs=-1, max_iter=1000)
        lr.fit(x_train, y_train)

        lr_train_acc = lr.score(x_train, y_train)
        lr_test_acc = lr.score(X_test,Y_test)

        lr_pr = precision_score(Y_test, lr.predict(X_test), average="weighted", zero_division=0)
        lr_rec = recall_score(Y_test, lr.predict(X_test), average="weighted", zero_division=0)
        lr_f1 = precision_recall_fscore_support(Y_test, lr.predict(X_test), average="weighted", zero_division=0)

        ovr_auroc = roc_auc_score(Y_test, lr.predict_proba(X_test), multi_class="ovr", average="weighted")
        ovo_auroc = roc_auc_score(Y_test, lr.predict_proba(X_test), multi_class="ovo", average="weighted")

        # Linear SVC
        l_svc = LinearSVC(dual=False)
        l_svc.fit(x_train, y_train)

        svc_train_acc = l_svc.score(x_train, y_train)
        svc_test_acc = l_svc.score(X_test, Y_test)

        svc_pr = precision_score(Y_test, l_svc.predict(X_test), average="weighted", zero_division=0)
        svc_rec = recall_score(Y_test, l_svc.predict(X_test), average="weighted", zero_division=0)
        svc_f1 = precision_recall_fscore_support(Y_test, l_svc.predict(X_test), average="weighted", zero_division=0)

        # Append performance metrics to lists
        lr_train_accs.append(lr_train_acc)
        lr_test_accs.append(lr_test_acc)
        svc_train_accs.append(svc_train_acc)
        svc_test_accs.append(svc_test_acc)
        lr_prs.append(lr_pr)
        lr_recals.append(lr_rec)
        lr_f1s.append(lr_f1)
        ovr_aurocs.append(ovr_auroc)
        ovo_aurocs.append(ovo_auroc)
        svc_prs.append(svc_pr)
        svc_recals.append(svc_rec)
        svc_f1s.append(svc_f1)

    # Calculate and return the mean values of the performance metrics
    return lr_train_accs, lr_test_accs, svc_train_accs, svc_test_accs, lr_prs, lr_recals, lr_f1s, svc_prs, svc_recals, svc_f1s, ovr_aurocs, ovo_aurocs


In [61]:
main_dict = {}
for year in tqdm(range(2011, 2021)):
    # pickle file for k-10 filings
    base_corpus=pickle.load(open(f"GPT/base/{year}_base_documents_embd.pkl","rb"))

    # labels csv file
    rev_df=pd.read_csv(f"labels/{year}.csv",index_col=1)

    performance_dict={'columns': ["lr_train_acc", "lr_test_acc", "svc_train_acc", "svc_test_acc", "lr_precision", "lr_recall", "lr_f1", "svc_precision", "svc_recall", "svc_f1", "OVO_auroc", "OVR_auroc"]}
    for i in range(int(year)+1,2023):
        try:
            performance_dict[i]=do_train_10f(base_corpus,i)
        except: print(f'error for {year} when doing {i}')
    main_dict[f"base_{year}"] = performance_dict

100%|██████████| 3650/3650 [00:00<00:00, 956168.23it/s]

100%|██████████| 3481/3481 [00:00<00:00, 931086.81it/s]

100%|██████████| 3673/3673 [00:00<00:00, 882442.35it/s]

100%|██████████| 3501/3501 [00:00<00:00, 878875.89it/s]

100%|██████████| 3638/3638 [00:00<00:00, 873733.28it/s]

100%|██████████| 3516/3516 [00:00<00:00, 887421.64it/s]

100%|██████████| 3664/3664 [00:00<00:00, 865896.43it/s]

100%|██████████| 3472/3472 [00:00<00:00, 872954.29it/s]

100%|██████████| 3682/3682 [00:00<00:00, 885314.57it/s]

100%|██████████| 3548/3548 [00:00<00:00, 882906.59it/s]

100%|██████████| 3675/3675 [00:00<00:00, 860064.01it/s]

100%|██████████| 3393/3393 [00:00<00:00, 851764.03it/s]

100%|██████████| 3653/3653 [00:00<00:00, 872291.06it/s]

100%|██████████| 403/403 [00:00<00:00, 664427.87it/s]

100%|██████████| 3701/3701 [00:00<00:00, 874147.94it/s]


error for 2011 when doing 2019



100%|██████████| 3610/3610 [00:00<00:00, 881289.65it/s]


error for 2011 when doing 2020



100%|██████████| 3681/3681 [00:00<00:00, 867177.77it/s]


...

error for 2019 when doing 2022



100%|██████████| 5418/5418 [00:00<00:00, 941528.80it/s]


error for 2020 when doing 2021



100%|██████████| 10/10 [19:11<00:00, 115.18s/it]

error for 2020 when doing 2022


In [62]:
import pickle

# Create a dictionary
my_dict = main_dict

# Specify the filename for the pickle file
pickle_filename = 'gpt_future_t5_.pkl'

# Open the file in binary write mode and use pickle.dump to save the dictionary
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(my_dict, pickle_file)

print(f'Dictionary has been pickled and saved to {pickle_filename}')

Dictionary has been pickled and saved to gpt_future_t5_.pkl
